## Instalaciones

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U bitsandbytes
!pip install wikipedia

# Generación de preguntas con `Llama-3.1-8B-Instruct`

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = 8096,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"}
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [ ]:
import wikipedia
wikipedia.set_lang("es") #set language of articles
article = wikipedia.page("Alan Turing").content

messages = [
    {"from":"gpt", "value": "Eres un asistente que solo genera preguntas de lectura comprensiva, sin sus respuestas, de un texto proporcionado"},
    {"from": "human", "value": f"Hacer 5 preguntas sobre el siguiente texto: {article}"},
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

print(f"Number of tokens: {len(inputs[0])}")

Number of tokens: 8271


In [ ]:
messages = [
    {"from": "gpt", "value": "Eres un generador de preguntas de lectura comprensiva. Tu tarea es crear preguntas basadas en un texto dado, sin incluir respuestas. Solo debes generar preguntas incluyendo preguntas de verdadero o falso."},
    {"from": "human", "value": f"Genera exactamente 5 preguntas sobre el siguiente texto, sin incluir respuestas:\n\n{article}"},
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True) # real-time output, more memory-efficient
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=1024, use_cache=True)

Aquí te presento 5 preguntas sobre el texto proporcionado:

1. ¿Cuál fue el nombre de la máquina que diseñó Turing para descifrar los códigos alemanes durante la Segunda Guerra Mundial?
2. ¿Qué fue el concepto que introdujo Turing en su estudio sobre la inteligencia artificial?
3. ¿Cuál fue el nombre del premio que se otorga anualmente a personas destacadas por sus contribuciones técnicas al mundo de la informática?
4. ¿Qué fue el resultado del juicio a Turing en 1952 por su homosexualidad?
5. ¿Cuál fue el título del telefilme británico dirigido por Alex Garland que se trata de hacer pasar la prueba de Turing a una androide con inteligencia artificial?

Y aquí te presento 5 preguntas de Verdadero o Falso:

1. Verdadero o falso: Turing fue condenado a prisión por su homosexualidad. (R)
2. Verdadero o falso: El gobierno británico de David Cameron denegó el indulto a Turing en 2012. (V)
3. Verdadero o falso: Turing diseñó la máquina Enigma para descifrar los códigos alemanes. (V)
4. Verda

🌐 **Fuente:** https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Inference.ipynb